In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'func': 'getbuyersguide',
                        'payload': '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[a, 'Type_Code'] + '","partkey":"' + input_.loc[a, 'Part_Key'] + '"}}}',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    pass
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for tr in list_tr:
                    list_td = [text.strip() for text in tr.xpath('./td/text()')]
                    list_vehicle.append(' '.join(list_td).strip())

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_TCPK': input_.loc[a, 'JOIN_TCPK'],
                                         'Vehcile': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Type_Code': input_.loc[a, 'Type_Code'],
                                     'Part_Key': input_.loc[a, 'Part_Key'],
                                     'JOIN_TCPK': input_.loc[a, 'JOIN_TCPK']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Type_Code'] + ' - ' + input_.loc[a, 'Part_Key'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Type_Code', 'Part_Key'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：310

[ok] - 4748 - 942614
[尝试次数：1] - [剩余数量：298] - [当前时间：18:23:43]

[ok] - 4748 - 937872
[尝试次数：3] - [剩余数量：297] - [当前时间：18:23:44]

[ok] - 4748 - 1188452
[尝试次数：2] - [剩余数量：296] - [当前时间：18:23:44]

[ok] - 4748 - 933539
[尝试次数：6] - [剩余数量：295] - [当前时间：18:23:45]

[ok] - 4748 - 944672
[尝试次数：1] - [剩余数量：294] - [当前时间：18:23:45]

[ok] - 4748 - 929301
[尝试次数：7] - [剩余数量：293] - [当前时间：18:23:45]

[ok] - 4748 - 929319
[尝试次数：2] - [剩余数量：292] - [当前时间：18:23:46]

[ok] - 4748 - 934202
[尝试次数：3] - [剩余数量：291] - [当前时间：18:23:46]

[ok] - 4748 - 1193592
[尝试次数：11] - [剩余数量：290] - [当前时间：18:23:47]

[ok] - 4748 - 932563
[尝试次数：2] - [剩余数量：289] - [当前时间：18:23:47]

[ok] - 4748 - 1185712
[尝试次数：4] - [剩余数量：288] - [当前时间：18:23:47]

[ok] - 4748 - 937194
[尝试次数：10] - [剩余数量：287] - [当前时间：18:23:47]

[ok] - 4748 - 937423
[尝试次数：1] - [剩余数量：286] - [当前时间：18:23:48]

[ok] - 4748 - 940325
[尝试次数：1] - [剩余数量：285] - [当前时间：18:23:48]

[ok] - 4748 - 1185107
[尝试次数：16] - [剩余数量：284] - [当前时间：18:23:49]

[ok] - 4748 - 943903
[尝试次数：1] - [剩余数量：283] - [当前时间：18:

[ok] - 4748 - 945228
[尝试次数：4] - [剩余数量：164] - [当前时间：18:24:27]

[ok] - 4748 - 942182
[尝试次数：13] - [剩余数量：163] - [当前时间：18:24:27]

[ok] - 4748 - 1190936
[尝试次数：1] - [剩余数量：162] - [当前时间：18:24:27]

[ok] - 4748 - 941349
[尝试次数：2] - [剩余数量：161] - [当前时间：18:24:28]

[ok] - 4748 - 944048
[尝试次数：2] - [剩余数量：160] - [当前时间：18:24:28]

[ok] - 4748 - 1192615
[尝试次数：11] - [剩余数量：159] - [当前时间：18:24:28]

[ok] - 4748 - 938560
[尝试次数：12] - [剩余数量：158] - [当前时间：18:24:29]

[ok] - 4748 - 940228
[尝试次数：5] - [剩余数量：157] - [当前时间：18:24:30]

[ok] - 4748 - 11553613
[尝试次数：1] - [剩余数量：156] - [当前时间：18:24:30]

[ok] - 4748 - 930373
[尝试次数：15] - [剩余数量：155] - [当前时间：18:24:30]

[ok] - 4748 - 933561
[尝试次数：3] - [剩余数量：154] - [当前时间：18:24:31]

[ok] - 4748 - 931571
[尝试次数：2] - [剩余数量：153] - [当前时间：18:24:31]

[ok] - 4748 - 945619
[尝试次数：2] - [剩余数量：152] - [当前时间：18:24:31]

[ok] - 4748 - 1191692
[尝试次数：5] - [剩余数量：151] - [当前时间：18:24:32]

[ok] - 4748 - 1190218
[尝试次数：9] - [剩余数量：150] - [当前时间：18:24:32]

[ok] - 4748 - 1188511
[尝试次数：7] - [剩余数量：149] - [当前时间：18:24:32

[ok] - 4748 - 942758
[尝试次数：1] - [剩余数量：32] - [当前时间：18:25:07]

[ok] - 4748 - 934609
[尝试次数：4] - [剩余数量：31] - [当前时间：18:25:08]

[ok] - 4748 - 942508
[尝试次数：6] - [剩余数量：30] - [当前时间：18:25:08]

[ok] - 4748 - 944259
[尝试次数：18] - [剩余数量：29] - [当前时间：18:25:08]

[ok] - 4748 - 931500
[尝试次数：7] - [剩余数量：28] - [当前时间：18:25:09]

[ok] - 4748 - 930252
[尝试次数：10] - [剩余数量：27] - [当前时间：18:25:09]

[ok] - 4748 - 938014
[尝试次数：6] - [剩余数量：26] - [当前时间：18:25:09]

[ok] - 4748 - 938299
[尝试次数：3] - [剩余数量：25] - [当前时间：18:25:09]

[ok] - 4748 - 940817
[尝试次数：12] - [剩余数量：24] - [当前时间：18:25:09]

[ok] - 4748 - 934427
[尝试次数：15] - [剩余数量：23] - [当前时间：18:25:09]

[ok] - 4748 - 935394
[尝试次数：3] - [剩余数量：22] - [当前时间：18:25:09]

[ok] - 4748 - 935261
[尝试次数：6] - [剩余数量：21] - [当前时间：18:25:09]

[ok] - 4748 - 942954
[尝试次数：1] - [剩余数量：20] - [当前时间：18:25:10]

[ok] - 4748 - 945864
[尝试次数：1] - [剩余数量：19] - [当前时间：18:25:10]

[ok] - 4748 - 13046677
[尝试次数：1] - [剩余数量：18] - [当前时间：18:25:10]

[ok] - 4748 - 938047
[尝试次数：4] - [剩余数量：17] - [当前时间：18:25:10]

[ok] - 4748 - 9402